|<h2>Book:</h2>|<h1><a href="https://open.substack.com/pub/mikexcohen/p/llm-breakdown-16-tokenization-words" target="_blank">50 ML projects to understand LLMs</a></h1>|
|-|:-:|
|<h2>Project:</h2>|<h1><b>[30] Sentiment analysis with decision trees</b></h1>|
|<h2>Author:<h2>|<h1>Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h1>|

<br>

<i>Using the code without reading the book may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch

from tqdm import tqdm
from scipy.stats import ttest_ind

from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from transformers import BertTokenizer, BertModel
from datasets import load_dataset

In [ ]:
### matplotlib adjustments (commented lines are for dark mode)

# svg plots (higher-res)
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

plt.rcParams.update({
    # 'figure.facecolor': '#282a2c',
    # 'figure.edgecolor': '#282a2c',
    # 'axes.facecolor':   '#282a2c',
    # 'axes.edgecolor':   '#DDE2F4',
    # 'axes.labelcolor':  '#DDE2F4',
    # 'xtick.color':      '#DDE2F4',
    # 'ytick.color':      '#DDE2F4',
    # 'text.color':       '#DDE2F4',
    'axes.spines.right': False,
    'axes.spines.top':   False,
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold',
    'savefig.dpi':300
})

# **Part 1: Import BERT and dataset**

In [ ]:
# load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertModel.from_pretrained('bert-large-uncased', output_hidden_states=True)
model.eval()

In [ ]:
# move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device);

In [ ]:
# load a subset (5%) of sst-2 sentiment dataset
dataset = load_dataset('glue','sst2',split='train[:5%]')

dataset

In [ ]:
dataset[123]

In [ ]:
# extract and count the labels
labels = [sample['label'] for sample in dataset]

num_samples = len(labels)

uniq,counts = np.unique(labels,return_counts=True)
for u,c in zip(uniq,counts):
  print(f'"{u}" appeared {c} times ({c/num_samples:.1%}).')

# **Part 2: Create batches**

In [ ]:
batchsize = 32
num_batch = num_samples//batchsize
sample_size = batchsize * num_batch

print(f'There are {num_batch} batches of size {batchsize}, leading to {sample_size} total samples.')

In [ ]:
allbatches = []
alllabels = []

for batchi in range(num_batch):

  # start and end indices
  startidx = batchi*batchsize
  endidx = startidx + batchsize

  # append texts
  tmp_texts = []
  tmp_labels = []
  for samplei in range(startidx,endidx):
    tmp_texts.append(dataset[samplei]['sentence'])
    tmp_labels.append(dataset[samplei]['label'])

  # tokenize
  tokens = tokenizer(tmp_texts,return_tensors='pt',padding=True)
  allbatches.append( tokens )
  alllabels.append( tmp_labels )

In [ ]:
len(alllabels), len(allbatches)

In [ ]:
len(alllabels[10]), allbatches[10], allbatches[10]['input_ids'].shape

In [ ]:
# lengths of different batches
seqlens = [allbatches[i]['input_ids'].shape[1] for i in range(len(allbatches))]
numseqs = [allbatches[i]['input_ids'].shape[0] for i in range(len(allbatches))]

plt.figure(figsize=(8,3))
plt.plot(numseqs,'ks',markerfacecolor=[.7,.7,.9],label='Number of sequences')
plt.plot(seqlens,'ko',markerfacecolor=[.7,.9,.7],label='Number of tokens')
plt.gca().set(xlabel='Batch number',ylabel='Counts',ylim=[10,70])
plt.legend()

plt.tight_layout()
plt.savefig('ch5_proj30_part2.png')
plt.show()

In [ ]:
# just one batch to confirm shapes
outputs = model(**allbatches[10].to(device))
outputs.hidden_states[4].shape

# **Part 3: Get the [CLS] hidden state activations from one layer**

In [ ]:
# this code block takes a very long time on a standard CPU, ~5 mins on a high-power CPU, and a few seconds on a GPU
cls_activations = []

for batchi in tqdm(range(num_batch)):

  # run the model
  outputs = model(**allbatches[batchi].to(device))

  # extract the CLS activation from the hidden states
  cls_batch = outputs.hidden_states[-1][:,0,:].detach().cpu().numpy()

  # append to a list
  cls_activations.append(cls_batch)

# and convert to numpy
cls_activations = np.vstack(cls_activations)
labels = np.hstack(alllabels)

cls_activations.shape, labels.shape

In [ ]:
# extract average activations
acts0 = cls_activations[labels==0,:].mean(axis=1)
acts1 = cls_activations[labels==1,:].mean(axis=1)

# t-test
t = ttest_ind(acts0,acts1)

# the violin plot
plt.figure(figsize=(8,5))
v = plt.violinplot([acts0,acts1])

# change the colors
v['bodies'][0].set_facecolor([.9,.7,.7])
v['bodies'][1].set_facecolor([.7,.9,.7])
v['cbars'].set_edgecolor('k')
v['cmins'].set_edgecolor('k')
v['cmaxes'].set_edgecolor('k')

# draw all the dots
plt.plot(np.random.normal(1,.03,len(acts0)),acts0,'.',color=[.9,.7,.7,.3])
plt.plot(np.random.normal(2,.03,len(acts1)),acts1,'.',color=[.7,.9,.7,.3])

# and finishing touches
plt.axhline(0,linestyle='--',linewidth=.2,color='k')
plt.gca().set(xticks=[1,2],xticklabels=['Negative','Positive'],ylabel='Mean [CLS] Activation',
              xlim=[.5,2.5],title=f'Mean [CLS] acts (t={t.statistic:.2f}, p={t.pvalue:.3f})')

plt.tight_layout()
plt.savefig('ch5_proj30_part3.png')
plt.show()

# **Part 4: Single-layer decision-tree classifier**

In [ ]:
# dimension reduction with pca
pca = PCA(n_components=.8)
act_reduced = pca.fit_transform(cls_activations)

print(f'Kept {pca.n_components_} components explaining {pca.explained_variance_ratio_.sum():.2%} variance.\n')
print(f'Original data size is {cls_activations.shape}')
print(f' Reduced data size is {act_reduced.shape}\n')
print(f'Observations:features ratio is {act_reduced.shape[0]}:{act_reduced.shape[1]} = {act_reduced.shape[0]/act_reduced.shape[1]:.1f}')

In [ ]:
# split the data into train and test
X_train,X_test, y_train,y_test = train_test_split(act_reduced,labels,test_size=.2,stratify=labels)
dectree = DecisionTreeClassifier(max_depth=4)
dectree.fit(X_train, y_train)

# train and test accuracy
train_acc = (dectree.predict(X_train) == y_train).mean()
test_acc  = (dectree.predict(X_test) == y_test).mean()

print(f'Accuracies: Train {train_acc:.2%}, Test: {test_acc:.2%}')

In [ ]:
# how many times to repeat the random data split
num_reps = 20

# re-initialize as arrays
train_acc = np.zeros(num_reps)
test_acc = np.zeros(num_reps)

# loop over reps
for i in range(num_reps):

  # run the analysis
  X_train,X_test,y_train,y_test = train_test_split(act_reduced,labels,test_size=.2,stratify=labels)
  dectree = DecisionTreeClassifier(max_depth=4)
  dectree.fit(X_train,y_train)

  # calculate and store accuracies
  train_acc[i] = (dectree.predict(X_train) == y_train).mean()
  test_acc[i]  = (dectree.predict(X_test) == y_test).mean()


In [ ]:
plt.figure(figsize=(10,4))

plt.plot(train_acc,'ko',markersize=12,markerfacecolor=[.7,.7,.9],label='Train')
plt.plot(test_acc,'ks',markersize=12,markerfacecolor=[.9,.7,.7],label='Test')

plt.axhline(train_acc.mean(),linestyle='--',color=[.7,.7,.9])
plt.axhline(test_acc.mean(),linestyle='--',color=[.9,.7,.7])

plt.gca().set(ylim=[.6,.9],xticks=range(0,num_reps,2),xlabel='Run',ylabel='Accuracy',
              title='Variability due to random data splitting')

plt.legend()

plt.tight_layout()
plt.savefig('ch5_proj30_part4a.png')
plt.show()

In [ ]:
# feature importances (of final run from previous cell)
importances = dectree.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,3))
plt.title("Feature Importances (Top 10 PCs)")
for i in range(10):
  imp_val = importances[indices[i]]
  plt.bar(i,imp_val,color=plt.cm.plasma(imp_val/importances.max()),edgecolor='k')
plt.xticks(range(10), [f"PC{indices[i]+1}" for i in range(10)], rotation=45)

plt.tight_layout()
plt.savefig('ch5_proj30_part4b.png')
plt.show()

In [ ]:
# FYI (not part of the exercise)
# The tree plot shows which features were used in each leaf split.
# However, this plot isn't really interpretable for PCA data.
import sklearn.tree
plt.figure(figsize=(14,10))
sklearn.tree.plot_tree(dectree,fontsize=10);

# **Part 5: Collect [CLS] activations from all layers**

In [ ]:
# the number of hidden-states from the model output (transformers + 1)
num_layers = model.config.num_hidden_layers + 1

# list to store all the activations
hs_acts = [np.zeros((sample_size,model.config.hidden_size)) for _ in range(num_layers)]

# loop over all batches
idx = 0
for batchi in tqdm(range(num_batch)):

  # run the model
  outputs = model(**allbatches[batchi].to(device))

  # extract the CLS activation from the hidden states
  for hsi,hs in enumerate(outputs.hidden_states):
    hs_acts[hsi][idx:idx+batchsize] = hs[:,0,:].detach().cpu().numpy()
  idx += batchsize


In [ ]:
len(hs_acts), hs_acts[hsi].shape

In [ ]:
# visualize the vector averages
plt.figure(figsize=(10,4))

for hsi in range(len(hs_acts)):
  plt.plot(np.random.normal(hsi-.2,.03,(labels==0).sum()),
           hs_acts[hsi][labels==0,:].mean(axis=1),'ko',markerfacecolor=[.9,.7,.7,.3],linewidth=0)
  plt.plot(np.random.normal(hsi+.2,.03,(labels==1).sum()),
           hs_acts[hsi][labels==1,:].mean(axis=1),'ks',markerfacecolor=[.7,.9,.7,.3],linewidth=.1)


plt.gca().set(xlabel='Hidden layer',ylabel='CLS activation (mean)')

plt.tight_layout()
plt.savefig('ch5_proj30_part5a.png')
plt.show()

# **Part 6: Laminar profile of classification accuracy**

In [ ]:
# main analysis

# initializations
accuracies = np.zeros((num_layers,2))

num_reps = 5
train_acc = np.zeros(num_reps)
test_acc = np.zeros(num_reps)


# loop over layers
for layeri in range(num_layers):


  # loop over repetitions for stability
  for i in range(num_reps):

    # 1) split the data
    X_train,X_test,y_train,y_test = train_test_split(hs_acts[layeri],labels,test_size=.2,stratify=labels)

    # 2) fit the PCA on the train data
    pca = PCA(n_components=.8)
    X_train_pca = pca.fit_transform(X_train) # apply to train data

    # 3) apply the PCA transform to the test set
    X_test_pca  = pca.transform(X_test) # apply to test data

    # 4) fit the decision-tree model
    dectree = DecisionTreeClassifier(max_depth=4)
    dectree.fit(X_train_pca,y_train)

    # train accuracy
    train_acc[i] = (dectree.predict(X_train_pca) == y_train).mean()
    test_acc[i]  = (dectree.predict(X_test_pca) == y_test).mean()

  # accuracies for this layer
  accuracies[layeri,0] = train_acc.mean()
  accuracies[layeri,1] = test_acc.mean()

  print(f'Finished layer {layeri+1:2}/{num_layers} with {accuracies[layeri,1]:.1%} test accuracy.')

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(accuracies[:,0],'ks-',linewidth=.2,markerfacecolor=[.7,.9,.7],markersize=12,label='Train')
plt.plot(accuracies[:,1],'ko-',linewidth=.2,markerfacecolor=[.7,.7,.9],markersize=12,label='Test')

plt.legend()
plt.gca().set(xlabel='Hidden state layer',ylabel='Accuracy')

plt.tight_layout()
plt.savefig('ch5_proj30_part5b.png')
plt.show()

# **Part 7: Performance benefit per transformer**

In [ ]:
# create the predictor variable (IV)
predictor = np.arange(1,num_layers).reshape(-1,1)

# fit the model
reg = LinearRegression(fit_intercept=True).fit(predictor,accuracies[1:,1])
print(f'beta_0: {reg.intercept_:6.3%}')
print(f'beta_1: {reg.coef_[0]:7.3%}')

# calculate predicted accuracies
yHat = reg.predict(predictor)

In [ ]:
# and visualize
plt.figure(figsize=(10,4))
plt.plot(predictor,yHat,'k',label='Linear fit')
plt.plot(predictor,accuracies[1:,1],'ko-',linewidth=.2,markerfacecolor=[.7,.7,.9],markersize=12,label='Data')

plt.gca().set(xlabel='Number of transformers',ylabel='Accuracy',
              title=f'Increase of {reg.coef_[0]:.2%} accuracy per transformer')
plt.legend(loc='upper left')

plt.tight_layout()
plt.savefig('ch5_proj30_part6.png')
plt.show()

In [ ]:
print(f'Regression-predicted boost per transformer  : {reg.coef_[0]:.4%}')
print(f'Empirically calculated boost per transformer: {np.mean(np.diff(accuracies[1:,1])):.4%}')